In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import datetime as dt
import alpaca_trade_api as tradeapi
import json
from alpaca_trade_api.rest import REST, TimeFrame
load_dotenv()

True

In [2]:
alpaca_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

tradeapi = REST(alpaca_key, alpaca_secret_key)

In [3]:
def get_daily_trade_data(ticker, start, end, tradeapi): 
    ticker_df = tradeapi.get_bars(
        ticker,
        TimeFrame.Day,
        start,
        end
    ).df
    ticker_df['ticker'] = ticker

    return ticker_df

In [4]:
get_daily_trade_data('HD', '2022-01-03', '2022-01-07', tradeapi)

,open,high,low,close,volume,trade_count,vwap,ticker
timestamp,,,,,,,,
2022-01-03 05:00:00+00:00,416.57,417.8446,403.26,408.64,3720356,85021,408.261522,HD
2022-01-04 05:00:00+00:00,408.64,413.6100,408.19,412.84,3264609,67397,411.849521,HD
2022-01-05 05:00:00+00:00,414.00,414.2200,406.81,407.24,4419021,84372,409.332968,HD
2022-01-06 05:00:00+00:00,408.56,409.0700,403.02,405.76,3307417,68950,405.615925,HD
2022-01-07 05:00:00+00:00,403.36,404.0900,392.88,393.61,4514133,89211,395.971181,HD


In [5]:
ticker_list = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')['Ticker'].to_list()

def make_tickers_df(ticker_list, start_date_str, end_date_str, tradeapi):
    ticker_dfs_list = [get_daily_trade_data(ticker, start_date_str, end_date_str, tradeapi) for ticker in ticker_list]
    tickers_df = pd.concat(ticker_dfs_list, axis=0, join='outer')
    tickers_df.index = tickers_df.index.date
    tickers_df = tickers_df[['ticker','close','volume']]
    return tickers_df
    
    #['symbol','close'].reset_index().rename(columns={'timestamp':'date'}).set_index('date').dt.date
    #return tickers_df.index.dt.date.reset_index().rename(columns={"timestamp":"date"})

In [6]:
stocks_df = make_tickers_df(ticker_list, '2012-06-01', '2022-06-01', tradeapi)
stocks_df.head()

,ticker,close,volume
2015-12-01,NFLX,125.37,12550755
2015-12-02,NFLX,128.93,24105712
2015-12-03,NFLX,126.81,26644540
2015-12-04,NFLX,130.93,20250411
2015-12-07,NFLX,125.36,23351531


In [7]:
def make_tickers_dict(ticker_list, start_date_str, end_date_str, tradeapi):
    return {ticker: get_daily_trade_data(ticker, start_date_str, end_date_str, tradeapi) for ticker in ticker_list}

In [8]:
ticker_list = ['AAPL', 'XOM']
my_tick_dict = make_tickers_dict(ticker_list, '2022-01-03', '2022-01-07', tradeapi)
my_tick_dict

{'AAPL':                              open    high     low    close     volume  \
 timestamp                                                               
 2022-01-03 05:00:00+00:00  177.81  182.88  177.71  182.010  104690466   
 2022-01-04 05:00:00+00:00  182.61  182.94  179.12  179.700   99284337   
 2022-01-05 05:00:00+00:00  179.61  180.17  174.65  174.835   86753815   
 2022-01-06 05:00:00+00:00  172.40  175.30  171.64  172.000   96792639   
 2022-01-07 05:00:00+00:00  172.73  174.14  171.03  172.170   87682425   
 
                            trade_count        vwap ticker  
 timestamp                                                  
 2022-01-03 05:00:00+00:00       772367  181.415512   AAPL  
 2022-01-04 05:00:00+00:00       831619  180.555450   AAPL  
 2022-01-05 05:00:00+00:00       844519  177.497784   AAPL  
 2022-01-06 05:00:00+00:00       958689  173.079885   AAPL  
 2022-01-07 05:00:00+00:00       716119  172.439116   AAPL  ,
 'XOM':                             open    

## News Article Vader Analyzer

In [17]:
from libs.drew_lib import form_df

from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\altma\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [18]:
api_key = os.getenv("NEWSAPI_KEY")
newsapi = NewsApiClient(api_key=api_key)
print(type(api_key))

<class 'str'>


In [19]:
company_df = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')
company_df = company_df.rename(columns={'Company Name': 'Company_Name'})
company_dict = dict(zip(company_df.Company_Name, company_df.Ticker))
company_list = list(company_dict.keys())
tickers_list = company_dict.values()
ls = [type(item) for item in company_list]
print(ls)
tickers_list


[<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>]


dict_values(['NFLX', 'FB', 'UBER', 'MCHP', 'ABNB', 'FANG', 'MRO', 'DVN', 'SPWR', 'REGI', 'MTRX', 'BLK', 'PYPL', 'MELI', 'SOFI'])

In [20]:
netflix_news_df = form_df('Netflix')
netflix_news_df

,date,ticker,text
0,2022-05-19,NFLX,Picking out something new to watch from Netfli...
1,2022-05-14,NFLX,The platform could add the capability for come...
2,2022-06-02,NFLX,The streaming wars are no longer about which a...
3,2022-05-10,NFLX,Streaming is a curious beast. One minute you'l...
4,2022-05-10,NFLX,The timetables are accelerating\r\nIllustratio...
5,2022-05-26,NFLX,Aloy could be on her way to Netflix.\r\n\n \n\...
6,2022-05-22,NFLX,
7,2022-05-28,NFLX,"Over a three-hour dinner, Mr. Sarandos was cha..."
8,2022-05-08,NFLX,"Ncuti Gatwa, a star of the Netflix series Sex ..."
9,2022-05-18,NFLX,Alongside 150 full time employees Netflix axed...


In [ ]:
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\altma\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
'''''''''''''''
def vader(df):
    company_list = company_dict.keys()

    df_list = []
    for company in company_list:
        try:
            df_list.append(form_df(company))
        except:
            continue
    news_df = pd.concat(df_list, axis=0, join='outer')
    
    return news_df



In [44]:
def vader_analyzer(df):
    analyzer = SentimentIntensityAnalyzer()
    df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df['text']]
    df['neg'] = [analyzer.polarity_scores(x)['neg'] for x in df['text']]
    df['neu'] = [analyzer.polarity_scores(x)['neu'] for x in df['text']]
    df['pos'] = [analyzer.polarity_scores(x)['pos'] for x in df['text']]

    df['date'] = pd.to_datetime(
    df['date'],
    infer_datetime_format = True,
    utc = True    
    )
    df['date'] = df['date'].dt.date
    
    return df



In [ ]:
'''''''''
def daily_sentiment(df):
    df = vader_analyzer(df)

    df['date'] = pd.to_datetime(
    df['date'],
    infer_datetime_format = True,
    utc = True    
    )
    
    return df.groupby('date')['compound','pos','neu','neg'].mean()


# date to datetime

    df['date'] = pd.to_datetime(
    df['date'],
    infer_datetime_format = True,
    utc = True    
    )
    df['date'] = df['date'].dt.date


In [45]:
df = vader_analyzer(netflix_news_df)
df

,date,ticker,text,compound,pos,neu,neg
0,2022-05-19,NFLX,Picking out something new to watch from Netfli...,-0.1280,0.000,0.957,0.043
1,2022-05-14,NFLX,The platform could add the capability for come...,0.3612,0.077,0.923,0.000
2,2022-06-02,NFLX,The streaming wars are no longer about which a...,-0.0258,0.137,0.723,0.139
3,2022-05-10,NFLX,Streaming is a curious beast. One minute you'l...,0.4404,0.148,0.779,0.073
4,2022-05-10,NFLX,The timetables are accelerating\r\nIllustratio...,0.1280,0.075,0.865,0.061
5,2022-05-26,NFLX,Aloy could be on her way to Netflix.\r\n\n \n\...,0.1027,0.037,0.963,0.000
6,2022-05-22,NFLX,,0.0000,0.000,0.000,0.000
7,2022-05-28,NFLX,"Over a three-hour dinner, Mr. Sarandos was cha...",0.0772,0.092,0.823,0.085
8,2022-05-08,NFLX,"Ncuti Gatwa, a star of the Netflix series Sex ...",0.3400,0.064,0.936,0.000
9,2022-05-18,NFLX,Alongside 150 full time employees Netflix axed...,-0.7430,0.000,0.814,0.186


In [50]:
news_df_list = [form_df(company) for company in company_list]
vader_news_df_list = [vader_analyzer(df) for df in news_df_list]
vader_news_df = pd.concat(vader_news_df_list, axis=0, join='inner')
cols = ['date','ticker','text','pos','neg','neu','compound']
vader_news_df.columns = cols
vader_news_df.to_csv('../Notebooks/Data/Cleaned_Data/news_sentiment.csv')